In [8]:
import spacy
from spacy.matcher import Matcher
import re

nlp = spacy.load("en_core_web_sm")
# doc = nlp("cantata (smash)hazelnut incense coffee beans 600g mocha blend")
# doc1 = nlp("[sales cafe] hazelnut incense coffee beans( 200g 500g 1kg )")
# doc3 = nlp("sanjibyeolsaengdu santos, brazil NY2 1Bag 60kg")
# doc4 = nlp("espresso bold dark 1kg SRBC140FC0500A02 coffee beans")

In [33]:
from spacy import displacy

displacy.render(doc3)
# doc3[7].dep_, doc3[7], doc3[7].pos_
# doc3[6].dep_, doc3[6], doc3[6].pos_

In [37]:
# print(spacy.explain("npadvmod"))
# print(spacy.explain("nummod"))
# print(spacy.explain("DET"))
# print(spacy.explain("quantmod"))
print(spacy.explain("CD"))
# noun phrase as adverbial modifier
# numeric modifier
# determiner
# Looking at dependencies to make logic
for t in doc3:
    print(t,t.lemma_)

cardinal number
sanjibyeolsaengdu sanjibyeolsaengdu
santos santo
, ,
brazil brazil
NY2 NY2
1Bag 1bag
60 60
kg kg


In [94]:
def detect_weight(input_string):
    # input is doc type from spacy
    wanted_pos = ['NOUN', 'PROPN']
    for idx,t in enumerate(input_string):
        if t.lower_ in ["g", " g ", "kg", " kg ", "oz", " oz "]:
                # if t.pos_ == 'PROPN':
                #     return str(t)
                # elif t.pos_ == 'NOUN' and input_string[idx -1].pos_ == 'NUM':
                #     return str(input_string[idx -1]) + str(t)
                return str(input_string[idx-1]) + str(t)
    return "None"
for item in z:
    print(detect_weight(nlp(item)))
#     """cantata (smash)hazelnut incense coffee beans 600g/mocha blend
# Costa costa signature blend coffee bean 200G x 2
# starbucks veranda blend light smash coffee 567g 2 pack
# trader joe organic french roast 13oz(369g) x2 pack
# [blackvins]mandeling, indonesia coffee beans after ordering roasting 200g
# for restaurant prima vending machine cohabitation 1kg coffee prim business
# [117557]ethiopia special blend(500g/fine grinding/mr bin)
# [keopimandeulgi] kenya aa TOP 500g
# rabbah intensos dark roast coffee beans powder 340g 3 piece
# pauls ground coffee beans 865g coffee powder drip coffee
# pauls breakfast blend mild light roast ground coffee 720g
# indonesia sumatra mandeling 100g coffee beans roasting on the day
# [sales cafe] hazelnut incense coffee beans(200g/500g/1kg)
# espresso bold dark 1kg(SRBC140FC0500A02)coffee beans
# el salvador fancy 1kg(SROC120CI0500A02) gobgebunswae/gajeongyongeseupeureso/mocha pot(M)
# [coffee pilgrims] guatemala shb beans coffee/227g 1 rod(8oz 1 rod) /single origin
# brazil serado coffee beans 1kg roasting on the day
# sanjibyeolsaengdu santos, brazil NY2 1Bag(60kg)
# """

600g
200G
567g
13oz(369g
200g
1kg
blend(500g
500g
340g
865g
720g
100g
beans(200g
None
None
227g
1kg
1Bag(60kg


In [1]:
train = [('cantata smash hazelnut incense coffee beans 600g mocha blend', {'entities': [(44, 48, 'MEASURE')]}), ('Costa costa signature blend coffee bean 200G x 2', {'entities': [(40, 44, 'MEASURE')]}), ('starbucks veranda blend light smash coffee 567g 2 pack', {'entities': [(43, 47, 'MEASURE')]}), ('trader joe organic french roast 13oz 369g x2 pack', {'entities': [(32, 36, 'MEASURE'), (37, 41, 'MEASURE')]}), ('[blackvins]mandeling, indonesia coffee beans after ordering roasting 200g', {'entities': [(69, 73, 'MEASURE')]}), ('for restaurant prima vending machine cohabitation 1kg coffee prim business', {'entities': [(50, 53, 'MEASURE')]}), ('[117557]ethiopia special blend 500g fine grinding mr bin', {'entities': [(31, 35, 'MEASURE')]}), ('[keopimandeulgi] kenya aa TOP 500g', {'entities': [(30, 34, 'MEASURE')]}), ('rabbah intensos dark roast coffee beans powder 340g 3 piece', {'entities': [(47, 51, 'MEASURE')]}), ('pauls ground coffee beans 865g coffee powder drip coffee', {'entities': [(26, 30, 'MEASURE')]}), ('pauls breakfast blend mild light roast ground coffee 720g', {'entities': [(53, 57, 'MEASURE')]}), ('indonesia sumatra mandeling 100g coffee bea 200g 500g 1kg', {'entities': [(28, 32, 'MEASURE'), (44, 48, 'MEASURE'), (49, 53, 'MEASURE'), (54, 57, 'MEASURE')]}), ('espresso bold dark 1kg SRBC140FC0500A02 ns roasting on the day', {'entities': [(19, 22, 'MEASURE')]}), ('el salvador fancy 1kg SROC120CI0500A02 gobgebunswae gajeongyongeseupeureso mocha pot M', {'entities': [(18, 21, 'MEASURE')]}), ('[coffee pilgrims] guatemala shb beans coffee 227g 1 rod 8oz 1 rod single origin', {'entities': [(45, 49, 'MEASURE'), (56, 59, 'MEASURE')]}), ('brazil serado coffee beans 1kg roasting on the day', {'entities': [(27, 30, 'MEASURE')]}), ('sanjibyeolsaengdu santos, brazil NY2 1Bag 60kg', {'entities': [(42, 46, 'MEASURE')]})]

In [10]:
from spacy.matcher import Matcher
import re

nlp = spacy.load("en_core_web_sm")

def parse_train_data(doc):
    start_for_product_name = matcher(doc)[0][1]
    #unit_list = ['grams','g','kg','kilograms','mg','oz','lb','lbs','ml','l','liter','liters']
    # measure_detections = [(doc[start:end].start_char, doc[start:end].end_char, 'MEASURE') for idx, start, end in matcher(doc)]
    
    for idx, start, end in matcher(doc):
        start_idx_overall, end_idx_overall = doc[start:end].start_char, doc[start:end].end_char
        re_match = re.match('(\d+)',str(doc[start_idx_overall:end_idx_overall]))
        start_num, end_num = re_match.span()
        value_of_measure = (doc[start_num:end_num].start_char, doc[start_num:end_num].end_char, 'value_of_measure')
        unit_of_measure = (doc[end_num+1:end_idx_overall].start_char, doc[end_num+1:end_idx_overall].end_char, 'unit_of_measure')
        measure_detections = value_of_measure.extend(unit_of_measure)
    return (doc.text, {'entities': measure_detections})

def clean_text(input_text):
    input_text =  input_text.strip()
    # 
    input_text = re.sub(r"[\n\t<>()/]", " ", input_text)
    input_text = " ".join(input_text.split())
    
    return input_text

overall_pattern = [{'LIKE_NUM': True},
        {'LOWER':{'IN':['grams','g','kg','kilograms','mg','oz','lb','lbs','ml','l','liter','liters' ]}}
        ]
#unit_pattern = [{'LOWER': {'IN':['grams','g','kg','kilograms','mg','oz','lb','lbs','ml','l','liter','liters' ]}}]


input_string = """beans 600g/mocha blend
    200G x 2
    567g 2 pack
    trader joe organic french roast 13oz(369g) x2 pack
    [blackvins]mandeling, indonesia coffee beans after ordering roasting 200g"""

matcher = Matcher(nlp.vocab)

matcher.add("Measure", [overall_pattern])
#matcher.add("Measure", [unit_pattern])

input_text = [clean_text(input_text) for input_text in input_string.splitlines()] 
train_data = [parse_train_data(d) for d in nlp.pipe(input_text) if len(matcher(d)) >= 1]
print(train_data)

AttributeError: 'NoneType' object has no attribute 'span'

In [15]:
doc = nlp("beans 600g mocha blend")
for idx, start, end in matcher(doc):
    matcher = Matcher(nlp.vocab)
    overall_pattern = [{'LIKE_NUM': True},
        {'LOWER':{'IN':['grams','g','kg','kilograms','mg','oz','lb','lbs','ml','l','liter','liters' ]}}
        ]
    matcher.add("Measure", [overall_pattern])
    start_idx_overall, end_idx_overall = doc[start:end].start_char, doc[start:end].end_char
    re_match = re.match('(\d+)',str(doc[start_idx_overall:end_idx_overall]))
    print(str(doc[start_idx_overall:end_idx_overall]))
    #start_num, end_num = re_match.span()